In [1]:
import torch
import pickle

from pathlib import Path
from itertools import combinations
from matplotlib.colors import LogNorm

import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
import nglview as nv

blue = (70 / 255, 110 / 255, 250 / 255)

# Plumed

In [175]:
molecule = "2jof"

## Visualization

In [176]:
gro_path = f"./{molecule.upper()}/folded.gro"
gro_traj = md.load(gro_path)
print(gro_traj)

gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.add_representation("licorice", selection="protein")

gro_ca_atoms = [atom for atom in gro_traj.topology.atoms if atom.name == "CA"]
gro_ca_indices = [atom.index + 1 for atom in gro_ca_atoms]
print("Alpha carbon indices:", gro_ca_indices)

gro_view

<mdtraj.Trajectory with 1 frames, 272 atoms, 20 residues, and unitcells>
Alpha carbon indices: [5, 17, 27, 48, 58, 75, 99, 118, 128, 140, 147, 156, 168, 179, 190, 197, 223, 237, 251, 263]


NGLWidget()

In [177]:
pdb_path = f"./{molecule.upper()}/folded.pdb"
pdb_traj = md.load_pdb(pdb_path)
print(pdb_traj)

pdb_traj.center_coordinates()
pdb_view = nv.show_mdtraj(pdb_traj)
pdb_view.add_representation("licorice", selection="protein")

pdb_ca_atoms = [atom for atom in pdb_traj.topology.atoms if atom.name == "CA"]
pdb_ca_indices = [atom.index + 1 for atom in pdb_ca_atoms]
print("Alpha carbon indices:", pdb_ca_indices)

pdb_view

<mdtraj.Trajectory with 1 frames, 272 atoms, 20 residues, without unitcells>
Alpha carbon indices: [2, 16, 26, 47, 57, 74, 98, 117, 127, 139, 146, 153, 167, 178, 189, 196, 220, 234, 248, 262]


NGLWidget()

In [178]:
np.array(gro_ca_indices) == np.array(pdb_ca_indices)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [179]:
cnt = 0
for i in range(len(gro_ca_indices)):
    for j in range(i+1, len(gro_ca_indices)):
        print(f"d{cnt+1}: DISTANCE ATOMS={gro_ca_indices[i]},{gro_ca_indices[j]}")
        cnt += 1
        
print("\n")
print("ARG=" + ",".join(f"d{i+1}" for i in range(cnt)))

d1: DISTANCE ATOMS=5,17
d2: DISTANCE ATOMS=5,27
d3: DISTANCE ATOMS=5,48
d4: DISTANCE ATOMS=5,58
d5: DISTANCE ATOMS=5,75
d6: DISTANCE ATOMS=5,99
d7: DISTANCE ATOMS=5,118
d8: DISTANCE ATOMS=5,128
d9: DISTANCE ATOMS=5,140
d10: DISTANCE ATOMS=5,147
d11: DISTANCE ATOMS=5,156
d12: DISTANCE ATOMS=5,168
d13: DISTANCE ATOMS=5,179
d14: DISTANCE ATOMS=5,190
d15: DISTANCE ATOMS=5,197
d16: DISTANCE ATOMS=5,223
d17: DISTANCE ATOMS=5,237
d18: DISTANCE ATOMS=5,251
d19: DISTANCE ATOMS=5,263
d20: DISTANCE ATOMS=17,27
d21: DISTANCE ATOMS=17,48
d22: DISTANCE ATOMS=17,58
d23: DISTANCE ATOMS=17,75
d24: DISTANCE ATOMS=17,99
d25: DISTANCE ATOMS=17,118
d26: DISTANCE ATOMS=17,128
d27: DISTANCE ATOMS=17,140
d28: DISTANCE ATOMS=17,147
d29: DISTANCE ATOMS=17,156
d30: DISTANCE ATOMS=17,168
d31: DISTANCE ATOMS=17,179
d32: DISTANCE ATOMS=17,190
d33: DISTANCE ATOMS=17,197
d34: DISTANCE ATOMS=17,223
d35: DISTANCE ATOMS=17,237
d36: DISTANCE ATOMS=17,251
d37: DISTANCE ATOMS=17,263
d38: DISTANCE ATOMS=27,48
d39: DISTANCE 

# TICA coordinates & trjconv

In [ ]:
xtc_path = "/bioemu/opes/simulations/gtt/tica/debug/0.xtc"
analysis_dir = Path(f"/bioemu/opes/simulations/gtt/tica/debug/")
tpr_path = "/bioemu/opes/data/GTT/nvt_0.tpr"
# cmd = f"gmx trjconv -f {xtc_path} -o {analysis_dir}/0_debug.xtc -pbc nojump "
cmd = f"gmx trjconv -f {xtc_path} -s {tpr_path} -pbc nojump -o {analysis_dir}/0.xtc"
print(cmd)

gmx trjconv -f /bioemu/opes/simulations/gtt/tica/debug/0.xtc -s /bioemu/opes/data/GT/nvt_0.tpr -pbc nojump -o /bioemu/opes/simulations/gtt/tica/debug/0.xtc


In [ ]:
method="tica"
moelcule="1fme"
date="0908_152748"

print(f"python analysis_opes.py --config-name {method}-{molecule} date={date} analysis.skip_steps=0 analysis.unit_steps=1 +tags=['vast','debug']")

python analysis_opes.py --config-name tica-1fme date=0908_152748 analysis.skip_steps=0 analysis.unit_steps=1 +tags=['vast','debug']


# OPES results

In [174]:
xtc_path = "/home/shpark/prj-mlcv/lib/bioemu/opes/simulations/gtt/tica/debug/0.xtc"
gro_path = "/home/shpark/prj-mlcv/lib/bioemu/opes/data/GTT/folded.gro"
traj = md.load(xtc_path, top=gro_path)
view = nv.show_mdtraj(traj)
view.add_representation("licorice", selection="protein")
view.display()

NGLWidget()

In [ ]:
export GMX_CUDA_GRAPH=0
export CUDA_LAUNCH_BLOCKING=1
gmx mdrun -s md.tpr -deffnm md -ntmpi 1 -ntomp 1 -reprod -pme cpu
gmx mdrun -s ./data/1FME/nvt_0.tpr -deffnm simulations/1fme/tica/debug/0 -plumed simulations/1fme/tica/debug/0/plumed.dat -nsteps 1000000 -reseed 0 -ntomp 1 -bonded gpu -nb gpu -pme gpu -dlb no -reprod -pme cpu

In [92]:
gro_path = "/home/shpark/prj-mlcv/lib/bioemu/opes/data/1FME/boxed.gro"
traj = md.load(gro_path)
view = nv.show_mdtraj(traj)
view.add_representation("licorice", selection="protein")
view.display()

NGLWidget()

In [ ]:
# gmx grompp -f 1fme.mdp -c start_center.gro -p topol.top -o nvt_1.tpr -maxwarn 2
# gmx grompp -f 1fme_2.mdp -c start_center.gro -p topol.top -o nvt_2.tpr -maxwarn 2

In [ ]:
gmx editconf -f folded.gro -o boxed.gro -c -d 1.0 -bt cubic
gmx solvate -cp boxed.gro -cs spc216.gro -o solv.gro -p topol.top
gmx grompp -f ions.mdp -c solv.gro -p topol.top -o ions.tpr -maxwarn 1
gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -pname NA -nname CL -neutral -conc 0.065
13
gmx grompp -f 1fme_ions.mdp -c solv_ions.gro -p topol.top -o nvt_ion.tpr

In [ ]:
gmx grompp -f ions.mdp -c solv.gro -p topol.top -o ions.tpr
gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -nname CL -neutral

In [ ]:
# GTT

gmx editconf -f folded.gro -o boxed.gro -bt cubic -box 5.2 5.2 5.2 -c

gmx solvate -cp boxed.gro -cs spc216.gro -o solv.gro -p topol.top

gmx grompp -f ions.mdp -c solv.gro -p topol.top -o ions.tpr -maxwarn 1

gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -nname CL -neutral

gmx grompp -f em.mdp -c solv_ions.gro -p topol.top -o em.tpr

gmx mdrun  -deffnm em -v

gmx grompp -f gtt.mdp -c em.gro -p topol.top -o nvt_1.tpr

In [ ]:
# 1FME
gmx pdb2gmx -f folded.pdb -o folded.gro -p topol.top -ff charmm27 -water tip3p --ignh

gmx grompp -f em.mdp -c solv_ions.gro -p topol.top -o em.tpr -maxwarn 1

gmx mdrun -deffnm em -ntmpi 1 -nb gpu